In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time
from datetime import date
import datetime

In [2]:
# Create Dataset folder for Download result 
from os import mkdir, makedirs
DATASETS_FOLDER_PATH = "Datasets" #-%s" % date.today().strftime("%Y%m%d")
makedirs(DATASETS_FOLDER_PATH)

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'Datasets'

In [3]:
def download_data(PREFIX = "GSM", ID_LIST = list(range(2987694, 2987923 + 1)), ATTRIBUTES = ["ID", "Title", "self-administration", "challenge"]):

    dfObj = pd.DataFrame([], columns = ATTRIBUTES)

    for id in tqdm(ID_LIST, ncols=134):
        web_url = "https://www.ncbi.nlm.nih.gov/biosample/%s" % PREFIX + str(id)
        r = requests.get(web_url)
        soup = BeautifulSoup(r.text, 'html.parser')
        trs = soup.find_all("tr") # tr has th and td
        appendObj = {}
        appendObj["ID"] = PREFIX + str(id)
        appendObj["Title"] = soup.find("title").text.split(" ")[0]

        for atr in ATTRIBUTES:

            for tr in trs:

                if tr.th.text == atr:
                    appendObj[atr] = tr.td.text
                    break
        dfObj = dfObj.append(appendObj, ignore_index=True)
    return(dfObj)

In [4]:
download_result = download_data()

  0%|                                                                                                         | 0/230 [00:00<?, ?it/s]C:\Users\TAEHYE~1\AppData\Local\Temp/ipykernel_11176/2649429810.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfObj = dfObj.append(appendObj, ignore_index=True)
  0%|▍                                                                                                | 1/230 [00:01<05:25,  1.42s/it]C:\Users\TAEHYE~1\AppData\Local\Temp/ipykernel_11176/2649429810.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfObj = dfObj.append(appendObj, ignore_index=True)
  1%|▊                                                                                                | 2/230 [00:03<06:25,  1.69s/it]C:\Users\TAEHYE~1\AppData\Local\Temp/ipykernel_11176/2649429810.py:21: FutureWarning

  9%|████████▊                                                                                       | 21/230 [00:31<04:44,  1.36s/it]C:\Users\TAEHYE~1\AppData\Local\Temp/ipykernel_11176/2649429810.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfObj = dfObj.append(appendObj, ignore_index=True)
 10%|█████████▏                                                                                      | 22/230 [00:32<04:49,  1.39s/it]C:\Users\TAEHYE~1\AppData\Local\Temp/ipykernel_11176/2649429810.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfObj = dfObj.append(appendObj, ignore_index=True)
 10%|█████████▌                                                                                      | 23/230 [00:34<04:45,  1.38s/it]C:\Users\TAEHYE~1\AppData\Local\Temp/ipykernel_11176/2649429810.py:21: FutureWarning

 18%|█████████████████▌                                                                              | 42/230 [01:02<04:12,  1.34s/it]C:\Users\TAEHYE~1\AppData\Local\Temp/ipykernel_11176/2649429810.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfObj = dfObj.append(appendObj, ignore_index=True)
 19%|█████████████████▉                                                                              | 43/230 [01:04<04:11,  1.35s/it]C:\Users\TAEHYE~1\AppData\Local\Temp/ipykernel_11176/2649429810.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfObj = dfObj.append(appendObj, ignore_index=True)
 19%|██████████████████▎                                                                             | 44/230 [01:05<04:13,  1.37s/it]C:\Users\TAEHYE~1\AppData\Local\Temp/ipykernel_11176/2649429810.py:21: FutureWarning

 27%|██████████████████████████▎                                                                     | 63/230 [01:32<03:51,  1.39s/it]C:\Users\TAEHYE~1\AppData\Local\Temp/ipykernel_11176/2649429810.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfObj = dfObj.append(appendObj, ignore_index=True)
 28%|██████████████████████████▋                                                                     | 64/230 [01:33<03:55,  1.42s/it]C:\Users\TAEHYE~1\AppData\Local\Temp/ipykernel_11176/2649429810.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfObj = dfObj.append(appendObj, ignore_index=True)
 28%|███████████████████████████▏                                                                    | 65/230 [01:34<03:41,  1.34s/it]C:\Users\TAEHYE~1\AppData\Local\Temp/ipykernel_11176/2649429810.py:21: FutureWarning

ConnectionError: HTTPSConnectionPool(host='www.ncbi.nlm.nih.gov', port=443): Max retries exceeded with url: /biosample/GSM2987767 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001D41BAB3D30>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [5]:
def post_modification_and_save_to_csv(dfObj, result_id = datetime.datetime.now().strftime("%H%M%S")):
    df = pd.read_csv("DATASETS/original.CSV")
    df_bi = pd.read_csv("DATASETS/Behavioral_index_data.csv")
    df_with_denoted_colnames = pd.DataFrame(data=[], index=df["Gene"])
    
    for i in df.columns[1:231]:
        challenge = str(dfObj.loc[dfObj['Title'] == i]["challenge"].values)[2:3]
        sa = str(dfObj.loc[dfObj['Title'] == i]["self-administration"].values)[2:3]

        region = i[0:3]
        sample_num = i[-3:]

        ai = str(df_bi.loc[df_bi['Animal.ID'] == int(sample_num)]["Addiction Index"].values[0])

#         if ai != "0.0":
        col_name = i[-3:len(i)] + "_" + i[0:3].upper() + "_"  + sa + challenge + "-" + ai
        df_with_denoted_colnames[col_name] = df[i].values
    df_with_denoted_colnames.to_csv(DATASETS_FOLDER_PATH + "/Total_Conditions_Counts_with_AI%s.csv" % result_id)

    df = pd.read_csv(DATASETS_FOLDER_PATH + "/Total_Conditions_Counts_with_AI%s.csv" % result_id)
    df_with_denoted_colnames = pd.read_csv(DATASETS_FOLDER_PATH + "/Total_Conditions_Counts_with_AI%s.csv" % result_id)


    regions = ["BLA", "VTA", "CPU", "HIP", "NAC", "PFC"] 

    for region in tqdm(regions, ncols=134, desc="for %s"%region):
        arr = []
        for col in df.columns[1:]:
            if col[4:7] == region:
                arr.append(col)
        df_tmp = pd.DataFrame(columns=arr)
        for s in arr:
            df_tmp[s] = df[s]
        df_tmp.index = df_with_denoted_colnames.Gene

    #     df_tmp.to_csv("Dataset_with_AI-20200216/%s_Count_Dataset.csv" % arr[0][4:7], index=True)
    #     df_tmp.to_csv("Dataset_with_AI-20200216/%s_Count_Dataset_no_index.csv" % arr[0][4:7], index=False)

        df_tmp.to_csv(DATASETS_FOLDER_PATH + "/%s_Count_Dataset%s.csv" % (arr[0][4:7], result_id), index=True)
        df_tmp.to_csv(DATASETS_FOLDER_PATH + "/%s_Count_Dataset_no_index%s.csv" % (arr[0][4:7], result_id), index=False)

In [6]:
post_modification_and_save_to_csv(dfObj = download_result, result_id="")

NameError: name 'download_result' is not defined